# Movies
___
Data source: [https://www.kaggle.com/stefanoleone992/imdb-extensive-dataset]

In [ ]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# read in data
movies_df = pd.read_csv('../../Data/Movies/kaggle/imdb-extensive-dataset/IMDb movies.csv')
names_df = pd.read_csv('../../Data/Movies/kaggle/imdb-extensive-dataset/IMDb names.csv')
ratings_df = pd.read_csv('../../Data/Movies/kaggle/imdb-extensive-dataset/IMDb ratings.csv')
tp_df = pd.read_csv('../../Data/Movies/kaggle/imdb-extensive-dataset/IMDb title_principals.csv')

In [ ]:
movies_df.shape

In [ ]:
movies_df.head()

In [ ]:
names_df.shape

In [ ]:
names_df.head()

In [ ]:
ratings_df.shape

In [ ]:
ratings_df.head()

In [ ]:
tp_df.shape

In [ ]:
tp_df.head()

___
## Movies DF - Cleaning

I'd like to get all the data I can regarding movies with a recorded budget in US dollars with accompaning box office grosses.

In [ ]:
movies_df.isnull().mean()

In [ ]:
# fixing the typo in the columns
movies_df.rename(columns={'worlwide_gross_income': 'worldwide_gross_income'}, inplace=True)

In [ ]:
movies_df.dtypes

In [ ]:
# create a new df with out the null values in budget and usa_gross_income
budgets_df = movies_df[(movies_df['budget'].notnull()) & (movies_df['usa_gross_income'].notnull())]
budgets_df['budget'].value_counts()

In [ ]:
budgets_df.shape

In [ ]:
budgets_df['budget'].map(lambda x: x.split()[0]).value_counts()

In [ ]:
# let's only look at movies with US dollars as the currencty
us_budget_df = budgets_df[budgets_df['budget'].map(lambda x: x.split()[0] == '$')]
us_budget_df.head()

In [ ]:
# converting budget, usa_gross_income, and worlwide_gross_income to numerical data
us_budget_df.loc[us_budget_df.index, 'budget'] = us_budget_df.loc[us_budget_df.index, 'budget'].map(
    lambda x: x.split()[-1]
).astype(int)


us_budget_df.loc[us_budget_df.index, 'usa_gross_income'] = us_budget_df.loc[
    us_budget_df.index, 'usa_gross_income'
].map(
    lambda x: x.lstrip('$')
).astype(int)


us_budget_df.loc[us_budget_df.index, 'worldwide_gross_income'] = us_budget_df.loc[
    us_budget_df.index, 'worldwide_gross_income'
].map(
    lambda x: x.lstrip('$')
).astype(int)

In [ ]:
us_budget_df.dtypes

In [ ]:
us_budget_df['usa_gross_income'].sort_values()

In [ ]:
us_budget_df.hist(figsize=(12,9));
plt.tight_layout();

After a quick cleaning of the budget/gross data, we can see from the histograms of those columns that the data is very heavily skewed right.
___
For a regression exercise let's take a look at making worldwide_gross_income our target.

In [ ]:
# first let's only look at the df where we have worldwide_gross_income
wwgross_df = movies_df.loc[movies_df['worldwide_gross_income'].notnull()]
wwgross_df.shape

In [ ]:
# here we're going to convert the worldwide_gross_income into numerical data starting with the USDs
usd_index = wwgross_df.loc[wwgross_df['worldwide_gross_income'].map(lambda x: x.split()[0]) == '$'].index
usd_values = wwgross_df.loc[usd_index, 'worldwide_gross_income'].map(lambda x: x.lstrip('$')).values

wwgross_df.loc[usd_index, 'worldwide_gross_income'] = usd_values

In [ ]:
wwgross_df['worldwide_gross_income'].map(lambda x: x.lstrip('$')).sort_values()

In [ ]:
# Now let's convert everything to USD. First let's do INR
inr_index = wwgross_df.loc[wwgross_df['worldwide_gross_income'].map(lambda x: x.split()[0] == 'INR')].index
inr_values = wwgross_df.loc[inr_index, 'worldwide_gross_income'].map(lambda x: x.lstrip('INR')).values

# exchange rate for 1 INR is equal to .013
inr_to_usd_values = [round(inr*.013).astype(str) for inr in inr_values.astype(int)]

wwgross_df.loc[inr_index, 'worldwide_gross_income'] = inr_to_usd_values

In [ ]:
wwgross_df.loc[inr_index].head()

In [ ]:
wwgross_df['worldwide_gross_income'].map(lambda x: x.lstrip('$')).sort_values()

In [ ]:
npr_index = wwgross_df.loc[wwgross_df['worldwide_gross_income'].map(lambda x: x.split()[0] == 'NPR')].index
npr_value = wwgross_df.loc[npr_index, 'worldwide_gross_income'].map(lambda x: x.split()[-1]).values

# exchange rate 1 npr to .0082 usd
npr_to_usd = npr_value.astype(int) * .0082

wwgross_df.loc[npr_index, 'worldwide_gross_income'] = npr_to_usd.astype(str)

In [ ]:
wwgross_df['worldwide_gross_income'].map(lambda x: x.lstrip('$')).sort_values()

In [ ]:
pkr_index = wwgross_df.loc[wwgross_df['worldwide_gross_income'].map(lambda x: x.split()[0] == 'PKR')].index
pkr_values = wwgross_df.loc[pkr_index, 'worldwide_gross_income'].map(lambda x: x.split()[-1]).values

# exchange rate 1 pkr to .0062
pkr_to_usd = pkr_values.astype(int) * .0062

wwgross_df.loc[pkr_index, 'worldwide_gross_income'] = pkr_to_usd.astype(str)

In [ ]:
wwgross_df['worldwide_gross_income'].map(lambda x: x.lstrip('$')).sort_values()

In [ ]:
# convert worldwide gross income to numerical
wwgross_df.loc[:,'worldwide_gross_income'] = wwgross_df['worldwide_gross_income'].map(
    lambda x: x.split('.')[0]
).astype(int)

In [ ]:
wwgross_df.dtypes

Next we should repeat the process for `budget` and `usa_gross_income`
 
The next step is to remove any extra columns we don't need when making predictions about `worldwide_gross_income` or `usa_gross_income`. Then I'm also going to drop all the nulls for now to get an initial model.  

Columns we are not going to need are:  
`imdb_title_id` (doesn't tell us anything relevant)  
`original_title` (nearly duplicate column, the `title` column is better)  
`date_published` (we're only interested in the year of release, not so much when it was published.)  
`description` (This is represented better in other columns like genre)

In [ ]:
wwgross_df.drop(columns=['imdb_title_id', 'original_title', 'date_published', 'description'], inplace=True)

In [ ]:
df = wwgross_df.dropna()
df.shape

In [ ]:
df.loc[df['budget'].map(lambda x: x.split()[0]) == 'EUR'].head()

In [ ]:
# get a list of all other budget currencies that we should convert
budget_currencies = df['budget'].loc[df['budget'].map(lambda x: x.split()[0] != '$')].map(
    lambda x: x.split()[0]).value_counts().index
budget_currencies

FRF, DEM, ESP, FIM, and ATS are obsolete may need to look at individually.  
  
in fact if we wanted to be more accurate we should adjust all monetary values for inflation, but for now we'll just convert what we have

In [ ]:
rates = [1.09, 1.24, .71, .64, .17, .0094, .013, .14, .097, .18, .00082, .15, .56, .13, .1, .0031, .041, .01, .031, .04,
 .71, .015, .013, 1.04, .24, .0026, .28, .18, .60, .018, .0012, .08, .0068]

# let's make a dictionary for the each currencies conversion rate to usd
conversion_dict = dict(zip(budget_currencies, rates))
conversion_dict

In [ ]:
# let's loop through each currency and convert them to usd
for cur in budget_currencies:
    # get indecies and values for all the same currency
    ind = df['budget'].loc[df['budget'].map(lambda x: x.split()[0] == cur)].index
    values = df.loc[ind, 'budget'].map(lambda x: x.split()[-1]).values
    
    # convert each value to usd
    to_usd = [("$ " + round(val*conversion_dict[cur]).astype(str)) for val in values.astype(int)]
    
    # update dataframe
    df.loc[ind, 'budget'] = to_usd

In [ ]:
# stripping away the '$'
usd_budget_index = df.loc[df['budget'].map(lambda x: x.split()[0] == "$")].index
usd_budget_values = df.loc[usd_budget_index, 'budget'].map(lambda x: x.lstrip('$')).values

df.loc[usd_budget_index, 'budget'] = usd_budget_values

In [ ]:
# update the dataframe
df['budget'] = df['budget'].astype(float).astype(int).values

In [ ]:
df.dtypes

In [ ]:
df.head()

In [ ]:
df['usa_gross_income'].map(lambda x: x.split()[0]).value_counts()

In [ ]:
df.loc[:,'usa_gross_income'] = df['usa_gross_income'].map(lambda x: x.split()[-1]).values.astype(int)

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
df.shape

Now all the currencies are all converted to USD and to numerical data and all the nulls have been removed. Now let's save out our cleaned df. 

In [ ]:
df.to_csv('../../Data/Movies/IMDB_Movies_cleaned.csv', index=False)